In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
API_KEY = api_key

In [3]:
client_id = client_id

In [4]:
headers = {'Authorization': 'Bearer %s' % API_KEY}
url='https://api.yelp.com/v3/businesses/search'

In [9]:
zip_codes_LA = list(np.arange(90001, 90104))

In [10]:
zip_codes_LA.append(90189)
list_remove = [90069, 90085, 90090, 90092, 90097, 90098, 90100]

for x in list_remove:
    zip_codes_LA.remove(x)

In [7]:
def get_values(key, list_dicts):
    values = []
    for dict_ in list_dicts:
        try:
            values.append(dict_[key])
        except:
            values.append(None)         
    return values

In [12]:
ids, names, rating, review_count, price, location, coordinates, transactions, categories, is_closed = [],[],[],[],[],[],[],[],[],[]
for zip_code in zip_codes_LA:
    params = {'term':'restaurants', 'location': str(zip_code), 'limit':'50'}
    req=requests.get(url, params=params, headers=headers)
    
    # proceed only if the status code is 200
    print('The status code is {}'.format(req.status_code))
    
    parsed = json.loads(req.text)
    try:
        businesses = parsed['businesses']


        ids.append(get_values('id', businesses))
        names.append(get_values('name', businesses))
        rating.append(get_values('rating', businesses))
        review_count.append(get_values('review_count', businesses))
        price.append(get_values('price', businesses))
        location.append(get_values('location', businesses))
        coordinates.append(get_values('coordinates', businesses))
        transactions.append(get_values('transactions', businesses))
        categories.append(get_values('categories', businesses))
        is_closed.append(get_values('is_closed', businesses))
    except:
        pass

The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status 

In [20]:
list_attrs = [ids, names, rating, review_count,
             price, transactions, is_closed]
name_attrs = ['ids', 'names', 'rating', 'review_count',
             'price', 'transactions', 'is_closed']

In [21]:
dict_flat = {}
for i, attrs in enumerate(list_attrs):
    dict_flat[name_attrs[i]] = [x for attr in attrs for x in attr]

In [22]:
alias_list = []
title_list = []

for category in categories:

    for cat in category:
        alias = []
        title = []
        for c in cat:
            try:
                alias.append(c["alias"])
                title.append(c["title"])
            except:
                pass
        alias_attr = ', '.join(alias)
        title_attr = ', '.join(title)

        alias_list.append(alias_attr) 
        title_list.append(title_attr) 

In [23]:
dict_flat['type_food'] = title_list
dict_flat['type_food_alias'] = alias_list

In [24]:
dict_flat['latitude'] = [x['latitude'] for coord in coordinates for x in coord]
dict_flat['longitude'] = [x['longitude'] for coord in coordinates for x in coord]

In [25]:
dict_flat['country'] = [x['country'] for loc in location for x in loc]
dict_flat['state'] = [x['state'] for loc in location for x in loc]
dict_flat['city'] = [x['city'] for loc in location for x in loc]
dict_flat['zip_code'] = [x['zip_code'] for loc in location for x in loc]

In [26]:
dict_flat.keys()

dict_keys(['ids', 'names', 'rating', 'review_count', 'price', 'transactions', 'is_closed', 'type_food', 'type_food_alias', 'latitude', 'longitude', 'country', 'state', 'city', 'zip_code'])

In [27]:
df = pd.DataFrame.from_dict(data=dict_flat)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4710 entries, 0 to 4709
Data columns (total 15 columns):
ids                4710 non-null object
names              4710 non-null object
rating             4710 non-null float64
review_count       4710 non-null int64
price              3809 non-null object
transactions       4710 non-null object
is_closed          4710 non-null bool
type_food          4710 non-null object
type_food_alias    4710 non-null object
latitude           4710 non-null float64
longitude          4710 non-null float64
country            4710 non-null object
state              4710 non-null object
city               4710 non-null object
zip_code           4710 non-null object
dtypes: bool(1), float64(3), int64(1), object(10)
memory usage: 519.8+ KB


In [30]:
def replace_empty_lists(list_entities):
    if len(list_entities) < 1:
        return None
    return list_entities

In [31]:
df['transactions_'] = df.transactions.apply(replace_empty_lists)

In [32]:
df.head()

,ids,names,rating,review_count,price,transactions,is_closed,type_food,type_food_alias,latitude,longitude,country,state,city,zip_code,transactions_
0,SlAhlIL5KAXb4136SHFkOw,Bangin Buns,4.0,5,None,"[delivery, pickup]",False,Fast Food,hotdogs,33.975267,-118.248158,US,CA,Los Angeles,90001,"[delivery, pickup]"
1,spKUIFIABLaU63Ex4PEYvQ,Rays BBQ,4.5,801,$$,[delivery],False,"Barbeque, Smokehouse, Salad","bbq, smokehouse, salad",33.986598,-118.229803,US,CA,Huntington Park,90255,[delivery]
2,hARUOlpi_buVjoU0gN2ugw,La Zona Sushi,5.0,7,None,"[delivery, pickup]",False,Sushi Bars,sushi,33.982125,-118.238637,US,CA,Los Angeles,90001,"[delivery, pickup]"
3,RYpJyi1RhScLZAYrqPMLKQ,Avila's El Ranchito Mexican Restaurant,4.5,666,$$,"[delivery, pickup]",False,Mexican,mexican,33.978496,-118.230545,US,CA,Huntington Park,90255,"[delivery, pickup]"
4,ACm6CB7VI0d-K5qNzEODhg,Lettuce Feast,4.0,206,$$,[],False,"Food Trucks, Vegan, Comfort Food","foodtrucks, vegan, comfortfood",33.978530,-118.249700,US,CA,Los Angeles,90001,None


In [33]:
df.to_csv('../../data/interim/restaurants/yelp_losAngeles.csv')